In [ ]:
from os import environ

from autogen import ConversableAgent, run
from autogen.messages.run_events import InputRequestEvent
from autogen.tools import tool

llm_config = {"config_list": [{"api_key": environ["OPENAI_API_KEY"], "model": "gpt-4o-mini"}]}

# Single agent run

In [ ]:
@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


agent = ConversableAgent(name="Alice", llm_config=llm_config, human_input_mode="TERMINATE")

response = run(
    agent,
    initial_message="What is the meaning of life? Call the appropriate function to find out",
    tools=[get_meaning_of_life],
)


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Two agent run

In [ ]:
agent_1 = ConversableAgent(name="Alice", llm_config=llm_config)
agent_2 = ConversableAgent(name="Bob", llm_config=llm_config)

response = run(agent_1, agent_2, initial_message="What is the meaning of life?")


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Multi agent run with chat manager

In [ ]:
from autogen.agentchat.groupchat.groupchat import GroupChatManager


def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


planner = ConversableAgent(
    name="Planner",
    system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    llm_config=llm_config,
)

reviewer = ConversableAgent(
    name="Reviewer",
    system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    llm_config=llm_config,
)

teacher = ConversableAgent(
    name="Teacher",
    system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    llm_config=llm_config,
)

teacher.register_for_llm()(submit_plan)
planner.register_for_execution()(submit_plan)
reviewer.register_for_execution()(submit_plan)

chat_manager = GroupChatManager(llm_config=llm_config)

response = run(
    planner,
    reviewer,
    teacher,
    initial_message="Create lesson plans for 4th grade.",
    chat_manager=chat_manager,
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Swarm chat manager

In [ ]:
from autogen import AfterWorkOption, ConversableAgent, LLMConfig
from autogen.agentchat.groupchat.chat_managers.swarm import SwarmChatManager

llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# 1. Create our agents
planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

lesson_planner = ConversableAgent(name="planner_agent", llm_config=llm_config, system_message=planner_message)

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Make sure you provide recommendations each time the plan is updated.
"""

lesson_reviewer = ConversableAgent(name="reviewer_agent", llm_config=llm_config, system_message=reviewer_message)

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

chat_manager = SwarmChatManager(llm_config=llm_config, max_rounds=10, after_work=AfterWorkOption.SWARM_MANAGER)

response = run(
    planner,
    reviewer,
    teacher,
    initial_message="Today, let's introduce our kids to the solar system.",
    chat_manager=chat_manager,
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)